In [1]:
from abc import ABC, abstractmethod
from datetime import datetime

import json

# Classes

#### Transacao

In [2]:
class Transacao(ABC):
    @property
    def valor(self):
        pass

    @abstractmethod
    def registrar(self, conta):
        self.conta = conta

#### Deposito

In [3]:
class Deposito(Transacao):
    def __init__(self, valor):
        self.__valor = valor

    @property
    def valor(self):
        return self.__valor
    
    def registrar(self, conta):
        sucesso = conta.depositar(self.valor)
        if sucesso:
            conta.historico.adicionar_transacao(self)

#### Saque

In [4]:
class Saque(Transacao):
    def __init__(self, valor):
        self.__valor = valor
    
    @property
    def valor(self):
        return self.__valor

    def registrar(self, conta):
        sucesso = conta.sacar(self.valor)
        if sucesso:
            conta.historico.adicionar_transacao(self)

#### Cliente

In [5]:
class Cliente():
    def __init__(self, endereco: str):
        self.__endereco = endereco
        self.__contas = []
    
    @property
    def endereco(self):
        return self.__endereco

    def realizar_transacao(self, conta, transacao):
        transacao.registrar(conta)

    def adicionar_conta(self, conta):
        pass

#### PessoaFisica *

In [6]:
class PessoaFisica(Cliente):
    def __init__(self, endereco: str, cpf: str, nome: str, data_nascimento):
        super().__init__(endereco)
        self.__cpf = cpf
        self.__nome = nome
        self.__data_nascimento = data_nascimento
    
    @property
    def cpf(self):
        return self.__cpf
    
    @property
    def nome(self):
        return self.__nome
    
    @property
    def data_nascimento(self):
        return self.__data_nascimento
    
    def __str__(self):
        return (f"""
            nome: {self.nome}
            cpf: {self.cpf}
            data_nascimento: {self.data_nascimento}
            endereco: {self.endereco}
        """)

#### Historico *

In [7]:
class Historico():
    def __init__(self) -> None:
        self.__transacoes = []
    
    @property
    def transacoes(self):
        return self.__transacoes

    def adicionar_transacao(self, transacao):
        self.__transacoes.append(
            {
                "tipo": transacao.__class__.__name__,
                "valor": transacao.valor,
                "data": datetime.now().strftime("%d-%m-%Y %H:%M:%S")
            }
        )

#### Conta *

In [8]:
class Conta():
    def __init__(self, numero_conta, cliente):
        self.__saldo = 0
        self.__numero_conta = numero_conta
        self.__agencia = "0001"
        self.__cliente = cliente
        self.__historico = Historico()

    @property
    def saldo(self):
        return self.__saldo
    
    @property
    def numero_conta(self):
        return self.__numero_conta
    
    @property
    def agencia(self):
        return self.__agencia
    
    @property
    def cliente(self):
        return self.__cliente
    
    @property
    def historico(self):
        return self.__historico
    
    @classmethod
    def nova_conta(cls, cliente, numero_conta):
        return cls(numero_conta, cliente)
    
    def sacar(self, valor):
        saldo = self.saldo
        excedeu_saldo = valor > saldo
        
        if valor > 0:
            if excedeu_saldo:
                print("Voce nao possui saldo suficiente!")
            else:
                self.__saldo -= valor
                print(f"Saque de R${valor:.2f} realizado com sucesso!")
                return True
        else:
            print("O valor para o saque é invalido!")
        return False
    
    def depositar(self, valor):
        if valor > 0:
            self.__saldo += valor
            print(f"Deposito de R${valor:.2f} realizado com sucesso!")
        else:
            print("O valor para o deposito é invalido!")
            return False
        return True

#### ContaCorrente *

In [9]:
class ContaCorrente(Conta):
    def __init__(self, numero_conta: int,  cliente: str, limite=500, limite_saques=3):
        super().__init__(numero_conta, cliente)
        self.__limite = limite
        self.__limite_saques = limite_saques

    def sacar(self, valor):
        qtd_saques = len(
            [transacao for transacao in self.historico.transacoes if transacao["tipo"] == Saque.__name__]
        )
        excedeu_limite = valor > self.__limite
        excedeu_saques = qtd_saques >= self.__limite_saques

        if excedeu_limite:
            print("O valor de saque excede o limite!")
        if excedeu_saques:
            print("Numero maximo de saques excedido!")
        else:
            return super().sacar(valor)
        
        return False
    
    def __str__(self):
        return (f"""
            numero_conta: {self.numero_conta}
            cliente: {self.cliente.nome}
            agencia: {self.agencia}
            saldo: {self.saldo}
            historico: {self.historico.transacoes}
        """)

# Funçoes

#### menu *

In [10]:
def menu():
    menu = """
===================== SISTEMA BANCARIO =====================
    [d]  Depositar
    [s]  Sacar
    [e]  Extrato
    [nc] Nova conta
    [lc] Listar contas
    [nu] Novo usuario
    [lu] Listar usuarios
    [q]  Sair
    => """
    return input(menu)

#### filtrar_clientes

In [11]:
def filtrar_clientes(cpf, clientes):
    cliente_filtrado = [cliente for cliente in clientes if cliente.cpf == cpf]
    return cliente_filtrado[0] if cliente_filtrado else None

#### filtrar_conta *

In [12]:
def filtrar_contas(numero_conta, contas, cliente):
    conta_filtrada = [conta for conta in contas if conta.cliente == cliente and conta.numero_conta == numero_conta]
    return conta_filtrada[0] if conta_filtrada else None


#### depositar *

In [13]:
def depositar(clientes, contas):
    cpf = input("Informe o CPF (somente numeros): ")

    if cpf.isdigit():
        cliente = filtrar_clientes(cpf, clientes)
        if not cliente:
            print("Cliente nao encontrado!")
            return
        
        numero_conta = input("Informe o numero da conta: ")

        if numero_conta.isdigit():
            numero_conta = float(numero_conta)
            conta = filtrar_contas(numero_conta, contas, cliente)
            if not conta:
                print("Conta nao encontrada!")
                return

            valor = float(input("Quanto deseja depositar? "))
            transacao = Deposito(valor)

            # cliente.realizar_transacao(conta, transacao)
            transacao.registrar(conta)
        else:
            print("Numero de conta invalido!")
    else:
        print("Numero de CPF invalido!")

#### sacar

In [14]:
def sacar(clientes, contas):
    cpf = input("Informe o CPF (somente numeros): ")

    if cpf.isdigit():
        cliente = filtrar_clientes(cpf, clientes)
        if not cliente:
            print("Cliente nao encontrado!")
            return
        
        numero_conta = input("Informe o numero da conta: ")

        if numero_conta.isdigit():
            numero_conta = float(numero_conta)
            conta = filtrar_contas(numero_conta, contas, cliente)
            if not conta:
                print("Conta nao encontrada!")
                return

            valor = float(input("Quanto deseja sacar? "))
            transacao = Saque(valor)

            # cliente.realizar_transacao(conta, transacao)
            transacao.registrar(conta)
        else:
            print("O numero da conta é invalido!")
    else:
        print("O numero do CPF é invalido!")

#### extrato_completo

In [15]:
def extrato_completo(clientes, contas):
    cpf = input("Informe o CPF (somente numeros): ")

    if cpf.isdigit():
        cliente = filtrar_clientes(cpf, clientes)
        if not cliente:
            print("Cliente nao encontrado!")
            return
        
        numero_conta = input("Informe o numero da conta: ")

        if numero_conta.isdigit():
            numero_conta = float(numero_conta)
            conta = filtrar_contas(numero_conta, contas, cliente)
            if not conta:
                print("Conta nao encontrada!")
                return
            
            print("\n======================= EXTRATO =======================")
            transacoes = conta.historico.transacoes

            if not transacoes:
                print("Nao houveram movimentaçoes nesta conta.")
            else:
                for transacao in transacoes:
                    print(f"Tipo: {transacao['tipo']} | Valor: {transacao['valor']} | Data: {transacao['data']}")
            print(f"Saldo: {conta.saldo:.2f}")
            
        else:
            print("Numero de conta invalido!")
    else:
        print("Numero de CPF invalido!")

#### gravar_transacao

In [16]:
def gravar_transacao(quantia, extrato):
    data = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    if opcao == "d":
        extrato.append([data, "Deposito", quantia])
    elif opcao == "s":
        extrato.append([data, "Saque", quantia])

#### criar_cliente *

In [17]:
def criar_cliente(clientes):
    cpf = input("Digite o CPF (somente números): ")
    cliente = filtrar_clientes(cpf, clientes)

    if cliente:
        print("Usuário já existe!")
        return False

    nome = input("Nome: ")
    data_nascimento = input("Data de nascimento: ")
    endereco = input("Endereço: (logradouro, nro - bairro - cidade/sigla estado) ")

    cliente = PessoaFisica(
        endereco = endereco, 
        cpf = cpf, 
        nome = nome, 
        data_nascimento = data_nascimento
    )
    clientes.append(cliente)

    print("Cliente criado com sucesso!")

#### criar_conta_corrente

In [18]:
def criar_conta_corrente(numero_conta, clientes, contas):
    cpf = input("Digite o CPF (somente números): ")
    cliente = filtrar_clientes(cpf, clientes)

    if not cliente:
        print("Usuário não encontrado!")
        return False

    conta = ContaCorrente.nova_conta(
        numero_conta = numero_conta,
        cliente = cliente
    )
    contas.append(conta)
    # cliente.contas.append(conta)
    
    print("Conta criada com sucesso!")

#### listar_clientes *

In [19]:
def listar_clientes(clientes):
    print("=================== CLIENTES ========================")

    for cliente in clientes:
        print(f"Titular: {cliente.nome} | CPF: {cliente.cpf} | Nascimento: {cliente.data_nascimento} | Endereço: {cliente.endereco}")


#### listar_contas

In [20]:
def listar_contas(contas):
    print("==================== CONTAS =========================")

    for conta in contas:
        print(f"Conta: {conta.numero_conta} | Cliente: {conta.cliente.nome} | Agencia: {conta.agencia} | Saldo: {conta.saldo}")

# Main() *

In [21]:
def main():

    clientes = []
    contas = []

    while True:
        
        opcao = menu()

        if opcao == "d":
            depositar(clientes, contas)
 
        elif opcao == "s":
            sacar(clientes, contas)

        elif opcao == "e":
            extrato_completo(clientes, contas)

        elif opcao == "nc":
            numero_conta = len(contas) + 1
            criar_conta_corrente(numero_conta, clientes, contas)

        elif opcao == "lc":
            listar_contas(contas)

        elif opcao == "nu":
            criar_cliente(clientes)
        
        elif opcao == "lu":
            listar_clientes(clientes)

        elif opcao == "q":
            break

        else:
            print('Operaçao invalida!')

In [23]:
main()

# Testes de Funcionalidade

#### Criaçao de cliente e conta

In [22]:
clientes = []
contas = []

In [ ]:
criar_cliente(clientes)

In [ ]:
for cliente in clientes:
    print(cliente)

In [ ]:
numero_conta = len(contas) + 1
criar_conta_corrente(
    numero_conta = numero_conta,
    clientes = clientes, 
    contas = contas,
)

In [ ]:
for conta in contas:
    print(conta)

#### Teste de deposito e saque

In [ ]:
depositar(clientes, contas)

In [ ]:
sacar(clientes, contas)

In [ ]:
extrato_completo(clientes)